We wanted to check the runtime for the ketu code in our configuration.

In [21]:
import swarced as sw
import time, pickle, random, os, run
import numpy as np
import multiprocessing as mp

In [131]:
qlist = os.listdir("/k2_data/query")

#No parallel 

In [63]:
qlist = os.listdir("/k2_data/query")
t = np.array([])
for q in random.sample(qlist,5):
    start = time.time()
    os.system("python run.py " + q[7:-6] + " 0 /k2_data/query/" + q)
    t = np.append(t,time.time()-start)

Without parallel processing each cycle took about 490 seconds with a standard deviation in a sample of 5 of 10 seconds. That means for 5 tasks it took about 40 minutes.

In [65]:
np.mean(t)

490.52086238861085

In [69]:
np.std(t)

9.9701895198145252

In [71]:
np.sum(t)/60.

40.876738532384238

#Multiprocessing 

Since that is so time intensive and we wish to run quite a few of these, we implemented multiprocessing. 

In [2]:
import multiprocessing as mp

In [4]:
qlist = os.listdir("/k2_data/query")
qlist_subset = random.sample(qlist,5)#since there are 158 files in that folder I just wanted to run 5

In [92]:
def single_run(qfile):
    start = time.time()
    #python run.py epicID campaign query_file_path
    #os.system("python run.py " + qfile[4:-2] + " 0 /k2_data/query/" + qfile)
    run.main([qfile[7:-6], 0, "/k2_data/query/" + qfile])
    return time.time() - start

In [28]:
ss = time.time()
pool = mp.Pool(processes=6)#this computer has 8 cores, I'm only going to use 6
results = pool.map(single_run, qlist_subset)
pool.close()
pool.join()
print(results)
ee = time.time()
ee-ss

[544.5501210689545, 584.634486913681, 546.0201680660248, 711.1573400497437, 644.8120350837708]
Querying Data
Querying Data
Querying Data
Querying Data
Querying Data
Finished querying Data in 2.31sFinished querying Data in 2.40sFinished querying Data in 2.43sFinished querying Data in 2.29sFinished querying Data in 2.27s




Querying Likelihood
Querying Likelihood
Querying Likelihood
Querying Likelihood
Querying Likelihood
tau = 0.947937011719tau = 0.947937011719tau = 0.947937011719tau = 0.902069091797tau = 0.942840576172




155 185
173 184
175 182
179 179
140 181
146 155
168 173
169 175
tau = 0.902069091797
127 140
142 146
174 168
171 169
tau = 1.15689086914144 127
137 142
174 174
171 171

142 144
140 137
tau = 0.947937011719
tau = 0.947937011719
306 306
142 142
142 140
tau = 1.62066650391tau = 0.795043945312tau = 1.15689086914
tau = 0.942840576172
142 142


tau = 5.69781494141tau = 0.973419189453tau = 0.947937011719
297 311
300 312


tau = 0.703308105469288 297
296 300
1658249 313

28

-711.1813549995422

The total runtime was 711 seconds or about 11 minutes.

In [31]:
ee-ss

711.1813549995422

Interestingly, each individual run varied more using multiprocessing.

In [30]:
results

[544.5501210689545,
 584.634486913681,
 546.0201680660248,
 711.1573400497437,
 644.8120350837708]

On average, they took 10 minutes each with a standard deviation of about a minute.

In [32]:
np.mean(results)/60

10.103913837273916

In [72]:
np.std(results)

63.865980797904115

This could be because the test was run on a different sample or because of some variable overhead time with multiprocessing.

In [71]:
qq

['EPIC202060911.q',
 'EPIC202073144.q',
 'EPIC202073440.q',
 'EPIC202073353.q',
 'EPIC202093968.q']

#Cacheing

This test the speed of cacheing versus not cacheing for a multiprocessing run. I anticipate the speed will be about the same, maybe slightly slower since data must be written. 

In [130]:
qq

['EPIC202060911.q',
 'EPIC202073144.q',
 'EPIC202073440.q',
 'EPIC202073353.q',
 'EPIC202093968.q']

In [96]:
qlist_subset = qq

In [123]:
reload(sw)
reload(run)

<module 'run' from 'run.py'>

In [124]:
def single_run(qfile):
    start = time.time()
    #python run.py epicID campaign query_file_path
    #os.system("python run.py " + qfile[4:-2] + " 0 /k2_data/query/" + qfile)
    run.main([qfile[4:-2], 0, "/k2_data/query_cache/" + qfile])
    return time.time() - start

In [125]:
ss = time.time()
pool = mp.Pool(processes=6)#this computer has 8 cores, I'm only going to use 6
results = pool.map(single_run, qq)
pool.close()
pool.join()
print(results)
ee = time.time()
ee-ss

[649.0818219184875, 614.6605250835419, 667.3451302051544, 609.237820148468, 640.0954308509827]
Querying Data
Querying Data
Querying Data
Querying Data
Querying Data
Finished querying Data in 1.17sFinished querying Data in 1.16sFinished querying Data in 1.33sFinished querying Data in 1.29sFinished querying Data in 1.16s




Querying LikelihoodQuerying LikelihoodQuerying LikelihoodQuerying LikelihoodQuerying Likelihood




tau = 0.902069091797tau = 0.947937011719tau = 0.947937011719tau = 0.942840576172tau = 0.947937011719




179 179
155 185
173 184
140 181
175 182
tau = 0.902069091797
146 155
168 173
127 140
169 175
tau = 1.15689086914142 146
174 168
144 127
171 169

137 142
174 174
142 144
171 171
306 306
140 137
tau = 0.947937011719
142 142
tau = 0.947937011719
tau = 1.15689086914
142 140
tau = 1.62066650391tau = 0.942840576172
tau = 0.795043945312tau = 5.69781494141142 142

tau = 0.973419189453

tau = 0.947937011719
297 311

300 312
1658tau = 0.703308105469288 297
249 313
296 300
 16

667.5064611434937

The time was slighlty longer with cacheing than without

In [126]:
ee-ss

667.5064611434937

In [127]:
results

[649.0818219184875,
 614.6605250835419,
 667.3451302051544,
 609.237820148468,
 640.0954308509827]

In [128]:
np.mean(results)/60

10.601402427355449